# Python 函數裝飾器

https://www.runoob.com/w3cnote/python-func-decorators.html

### 一切皆對象

In [3]:
# 首先理解 Python 中的函數

def hi(name="yasoob"):
    return "hi " + name

hi()

'hi yasoob'

In [4]:
# 將一個函數賦值給一個變量，比如:
greet = hi
# 這裡hi甚至沒有加上()，以你為我們並不是在調用hi函數
# 而是將它放在greet變量裡

greet()

'hi yasoob'

In [5]:
# 這時，如果我們刪掉舊的hi函數，看看會發生什麼？

del hi
hi()

NameError: name 'hi' is not defined

In [10]:
greet()

'hi yasoob'

---

### 在函數中定義函數

以上的為函數的基本只是，接下來更近一步

In [7]:
def hi(name="yasoob"):
    print("now you are inside the hi() function")
    
    def greet():
        return "now you are in the greet() function"
    
    def welcome():
        return "now youa re in the welcome() function"
    
    print(greet())
    print(welcome())
    print("now you are back in the hi() function")
    
# 無論何時你調用了hi(),greet()和welcome()將會同時被調用
hi()

now you are inside the hi() function
now you are in the greet() function
now youa re in the welcome() function
now you are back in the hi() function


In [13]:
# 然而greet()和welcome()函數在hi()函數之外是不能訪問的：
greet()

NameError: name 'greet' is not defined

---

### 從函數中返回函數

現在知道了可以在函數中定義另外的函數。也就是說，我們可以創建嵌套的函數。

其實並不需要在一個函數裡去執行另一個函數，我們也可以將其作為輸出返回出來。

In [14]:
def hi(name="yasoob"):
    def greet():
        return "now you are in the greet() function"
    
    def welcome():
        return "now you are in the welcome() funciton"
    
    if name == "yasoob":
        return greet
    else:
        return welcome
    
a = hi()

# 結果清晰的展示了`a`現在指向了hi()函數中的greet()函數 
print(a)

<function hi.<locals>.greet at 0x0000016BEF72FD30>


In [15]:
print(a())

now you are in the greet() function


透過再次看一遍這個代碼。在 if/else 語句中我們返回 greet 和 welcome，而不是 greet() 和 welcome()。是因為當你把一堆小括號放在後面，這個函數就會執行；然而如果你不放括號在它後面，那它可以被導出傳遞，並且可以賦值給別的變量而不去執行它。

當我們寫下 a = hi()，hi()會被執行，而由於 name 參數默認是 yasoob，所以 greet 被返回了。如果我們把語句改為 a = hi(name = "ali"),那麼 welcome 函數將會被返回。我們還可以打印出hi()()，這將會輸出 now you are in the greet() function.

In [19]:
# 首先呼叫了hi()的函數，再一個()則呼叫了hi()，name="yasoob"的函數。
hi()()

'now you are in the greet() function'

---

### 將函數作為參數傳給另一個函數

In [21]:
def hi():
    return "hi yasoob!"

def doSomethingBeforHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())
    
doSomethingBeforHi(hi)

I am doing some boring work before executing hi()
hi yasoob!


---

### 你的第一個裝飾器

裝飾器能讓你在一個函數的前後去執行代碼

透過將函數作為參數傳給另一個函數（裝飾器），我們修改上一個裝飾器，並編寫一個稍微更加有用的程序。

In [30]:
def a_new_decorator(a_func):
    
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        
        a_func()
        
        print("I am doing some boring work after executing a_func()")
        
    return wrapTheFunction

def a_function_requiring_decoration():
    print("I am the function which need some decoration to remove my foul smell")
    
a_function_requiring_decoration()

I am the function which need some decoration to remove my foul smell


In [28]:
a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which need some decoration to remove my foul smell
I am doing some boring work after executing a_func()


我們應用了之前學習到的原理，這正是 python 裝飾器做的事情。它們封裝了一個函數，並且用這樣或者那樣的方式來修改它的行為。使用 `@` 符號，可以以一個簡短的方式來生成一個被裝飾的函數，下面使用 `@` 來運行之前的代碼：

In [31]:
@a_new_decorator
def a_function_requiring_decoration():
    """Hey you! Decorate me!"""
    print("I am the function which needs some decoration to remove my foul smell")
    
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


In [ ]:
# the @a_new_decorator is just a short way of saying:
a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)

In [32]:
print(a_function_requiring_decoration.__name__)

wrapTheFunction


這並不是我們想要的結果. Output輸出應該是"a_function_requiring_decoration"，但這裡的函數被warpTheFunction替代了。它重寫了我們函數的名字和注釋文檔。

我們可以用 python 提供給我們一個簡單的函數來解決這個問題，那就是functools.wraps。接下來我們直接用functools.wraps來修改上一個例子：

In [33]:
from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
        
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")
    
print(a_function_requiring_decoration.__name__)

a_function_requiring_decoration


In [35]:
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


藍本規範：

In [36]:
from functools import wraps
def decorator_name(f):
    # @wraps接受一個函數來進行裝飾，並加入了複製函數名稱、注釋文檔、參數列表等等功能。這可以讓我們在裝飾漆裡面訪問在裝飾之前的函數的屬性。
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated
 
@decorator_name
def func():
    return("Function is running")
 
can_run = True
print(func())
# Output: Function is running
 
can_run = False
print(func())

Function is running
Function will not run


---

### 使用場景

接下來看一下裝飾器在哪些地方特別耀眼，以及使用它可以讓一些事情管理變得更加簡單。

<b>授權（Authorization）</b>

裝飾器能有助於檢查某個人是否被授權使用一個 web 應用的端點（endpoint）。它們被大量適用於 Flask 和 Django web 框架中。這裡一個例子來使用基於裝飾器的授權：

In [ ]:
from functools import warps

def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated

<b>日誌(Logging)</b>

日誌是裝飾器運用的另一個亮點。

In [4]:
from functools import wraps

def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging

@ logit
def addition_func(x):
    """Do some math."""
    return x + x

result = addition_func(4)
result

addition_func was called


8

---

### 帶參數的裝飾器

難道@wraps不也是個裝飾器嗎？但是，它接收一個參數，就像任何普通的函數能做的那樣。

當你是用@my_decorator語法時，你是在應用一個單個函數作為參數的一個包裹函數。記住，Python裡每個東西都是一個對象，而且這包括函數！記住了這些，就可以編寫一下返回一個包裹函數的函數。
<br><br>


<b>在函數中嵌入裝飾器</b>

我們回到日誌的例子，並創建一個包裹函數，能讓我們指定一個用於輸出的日誌文件。

In [5]:
from functools import wraps

def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            
            # 打開logfile，並寫入內容
            with open(logfile, 'a') as opened_file:
                # 現在將日誌打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator

@logit()
def myfunc1():
    pass

# 現在一個叫做 out.log 的文件出現了，裡面的內容就是上面的字符串
myfunc1()

myfunc1 was called


In [7]:
@logit(logfile='func2.log')
def myfunc2():
    pass

# 現在一個叫做 func2.log 的文件出現了，裡面的內容就是上面的字符串
myfunc2()

myfunc2 was called


---

### 裝飾器類

現在有了能用於正式環境的 logit 裝飾器，但當我們的應用的某些部分還比較脆弱時，異常也許是需要更緊急關注的事情。比方說有時你只想打日誌到一個文件，而有時你先把引起你注意的問題發送到一個 email，同時也保留日誌，留個記錄。這是一個使用繼承的場景，但目前為止我們只看到過用來構建裝飾器的函數。

幸運的是，類也可以用來構建裝飾器。那我們現在以一個類而不是一個函數的方式，來重新構建 logit。

In [8]:
from functools import wraps

class logit(object):
    def __init__(self, logfile = 'out.log'):
        self.logfile = logfile
        
    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            
            # 打開 logfile 並寫入
            with open(self.logfile, 'a') as opened_file:
                # 現在將日誌打到指定的文件
                opened_file.write(log_string + '\n')
            # 現在，發送一個通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function
    
    def notify(self):
        # logit 只打日誌，不做別的
        pass

In [9]:
# 這個實現有一個附加優勢，在於比嵌套函數的方式更加整潔，而且包裹一個函數函式使用跟以前一樣的語法：
@logit()
def myfunc1():
    pass

In [10]:
# 現在，我們給 logit 創建子類，來添加 email 的功能：

class email_logit(logit):
    '''
    一個logit的實現版本，可以在函數調用時發送email給管理員
    '''
    def __init__(self, email = 'admin@admin.com', *args, **kwargs):
        self.email = email
        super(email_logit, self).__init__(*args, **kwargs)
        
    def notify(self):
        # 發送一封email到self.email
        # 這裡就不做實現了
        pass

這樣嵌套`email_logit(logit)`兩個函式之後，將會產生同樣的效果。但在寫日誌的基礎上，還會多發送一封郵件給管理員。

在這個版本的 hi() 函數中，它接受一個函數作為參數 func，然後定義了一個新的函數 wrapper()，在這個函數中會先輸出一些文字，然後執行傳入的函數 func()，最後再輸出一些文字。

在這個版本中，當我們使用裝飾器 @hi 套用在 my_name() 函數上時，它會將 my_name() 函數作為參數傳入 hi() 函數中，然後返回一個新的函數 wrapper()，最後將 my_name() 函數重新指向這個新的函數 wrapper()，這樣當我們調用 my_name() 函數時，實際上會執行的是 wrapper() 函數。

In [1]:
def hi(func):
    def wrapper():
        print("Calling hi() function")
        func()
        print("Exiting hi() function")
    return wrapper

@hi
def my_name():
    value1 = "PAN"
    print(f"My name is {value1}")

my_name()

Calling hi() function
My name is PAN
Exiting hi() function
